In [1]:
from pk.Ownership import Ownership
import pandas as pd
import geopandas as gpd
import numpy as np
from pk.Djmod import Djlog,logErr
from shapely.geometry import MultiLineString,mapping,Point,LineString
Ow = Ownership(r'E:\工作文档\SLLJDJZD2.gdb')
boundary = Ow.jzd_boundary.copy()
boundary['ZDDM_INDEX'] = boundary.ZDDM.str.cat(boundary.INDEX.astype(str))
log = Djlog()
add_jzx_all = Ow.add_jzx_all()
for i in add_jzx_all:
    pass
    
JZXDF = Ow.JZX.copy()
JZXDF['ZDDM_INDEX'] = JZXDF.ZDDM.str.cat(JZXDF.INDEX.astype(str))

zd_node = pd.DataFrame(columns=('QLRMC','ZDDM','INDEX','ZDDM_INDEX','N','X','Y'))
for index,row in Ow.ZD.iterrows():
    geojson = mapping(row.geometry)
    n = 0
    for index,value in enumerate(geojson['coordinates'][0]):
        zd_node_row = [{'QLRMC':row.QLRMC,'ZDDM':row.ZDDM,'INDEX':index,'ZDDM_INDEX':row.ZDDM+str(index),'N':n+1,'X':coor[0],'Y':coor[1]} for n,coor in enumerate(value[:len(value)-1])]
        if not zd_node.shape[0]:
            zd_node = pd.DataFrame(zd_node_row)
        else:
            zd_node = pd.concat([zd_node,pd.DataFrame(zd_node_row)],ignore_index=True)
zd_node['int_X'] = np.round(zd_node.X*10).astype('int64')
zd_node['int_Y'] = np.round(zd_node.Y*10).astype('int64')


In [ ]:
@logErr(log)
def linedef():
    jzx = gpd.GeoDataFrame(columns=[*JZXDF.columns,'geometry'],crs=Ow.ZD.crs)
    line = []
    for index,value in JZXDF.iterrows():
        ZD_boundary = zd_node[zd_node.ZDDM_INDEX == value.ZDDM_INDEX].reset_index()
        q_jzd = Ow.JZD[(Ow.JZD.ZDDM ==value.ZDDM) & (Ow.JZD.JZD_NEW == value.QSDH)]
        z_jzd = Ow.JZD[(Ow.JZD.ZDDM ==value.ZDDM) & (Ow.JZD.JZD_NEW == value.ZZDH)]
        q_line_index = ZD_boundary[(ZD_boundary.int_X==q_jzd.X.values[0]) & (ZD_boundary.int_Y==q_jzd.Y.values[0])].index[0]
        z_line_index = ZD_boundary[(ZD_boundary.int_X==z_jzd.X.values[0]) & (ZD_boundary.int_Y==z_jzd.Y.values[0])].index[0]
        xy_list = []
        if q_line_index>z_line_index:
            max_index = np.max(ZD_boundary.index.values)
            q_line_df = ZD_boundary.iloc[q_line_index:max_index+1]
            h_line_df = ZD_boundary.iloc[0:z_line_index+1]
            xy_list = list(zip(list(q_line_df.X),list(q_line_df.Y))) + list(zip(list(h_line_df.X),list(h_line_df.Y)))
        elif q_line_index == z_line_index:
            xy_list = list(zip(list(ZD_boundary.X),list(ZD_boundary.Y)))
        else:  
            line_df = ZD_boundary[q_line_index:z_line_index+1]
            xy_list = list(zip(list(line_df.X),list(line_df.Y)))
        # jzx = pd.concat([jzx,gpd.GeoDataFrame({**value,'geometry':LineString(xy_list)},crs=Ow.ZD.crs)],ignore_index=True)
        if xy_list:
            log.info({**value,'xy_list':xy_list})
            jzx.loc[index] = [*[v for v in value.values],MultiLineString([xy_list])]
    return jzx
    jzx.to_file('jzx.shp',encoding='gb18030')

linejzx = linedef()
linejzx

In [17]:
linejzx

,ZDDM,QLRMC,LZQLRMC,QSDH,ZJDH,ZZDH,INDEX,BXZ,BCM,BSM,XLXZ,XLCM,XLSM,ZDDM_INDEX,geometry
0,500118002015JA10033,胜利路街道堂皇坝村冰糟沟村民小组,NaN,J1,[],J2,0,NaN,NaN,NaN,NaN,NaN,NaN,500118002015JA100330,"MULTILINESTRING ((35581443.309 3251392.421, 35..."
1,500118002015JA10033,胜利路街道堂皇坝村冰糟沟村民小组,胜利路街道堂皇坝村石马村民小组,J2,[J3],J4,0,NaN,NaN,NaN,NaN,NaN,NaN,500118002015JA100330,"MULTILINESTRING ((35582019.672 3251592.659, 35..."
2,500118002015JA10033,胜利路街道堂皇坝村冰糟沟村民小组,NaN,J4,[J5],J1,0,NaN,NaN,NaN,NaN,NaN,NaN,500118002015JA100330,"MULTILINESTRING ((35581783.845 3251401.322, 35..."
3,500118002015JA10122,胜利路街道堂皇坝村冰糟沟村民小组,NaN,J1,[],J2,0,NaN,NaN,NaN,NaN,NaN,NaN,500118002015JA101220,"MULTILINESTRING ((35582029.981 3251594.536, 35..."
4,500118002015JA10122,胜利路街道堂皇坝村冰糟沟村民小组,胜利路街道堂皇坝村石马村民小组,J2,"[J3, J4]",J5,0,NaN,NaN,NaN,NaN,NaN,NaN,500118002015JA101220,"MULTILINESTRING ((35582556.221 3251876.724, 35..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,500118002020JA10031,胜利路街道永钢村长石坝村民小组,胜利路街道永钢村染房村民小组,J20,[J21],J22,0,NaN,NaN,NaN,NaN,NaN,NaN,500118002020JA100310,"MULTILINESTRING ((35585312.611 3255927.897, 35..."
567,500118002020JA10031,胜利路街道永钢村长石坝村民小组,NaN,J22,[],J23,0,NaN,NaN,NaN,NaN,NaN,NaN,500118002020JA100310,"MULTILINESTRING ((35585350.404 3255922.116, 35..."
568,500118002020JA10031,胜利路街道永钢村长石坝村民小组,胜利路街道永钢村涂家岩村民小组,J23,[],J1,0,NaN,NaN,NaN,NaN,NaN,NaN,500118002020JA100310,"MULTILINESTRING ((35585201.844 3256032.469, 35..."
569,500118002020JA10031,胜利路街道永钢村长石坝村民小组,NaN,J24,"[J25, J26]",J24,1,NaN,NaN,NaN,NaN,NaN,NaN,500118002020JA100311,"MULTILINESTRING ((35585435.574 3256147.955, 35..."


In [16]:
linejzx.to_file('jzx.shp',encoding='utf-8')

ValueError: Invalid field type <class 'list'>